In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
import pandas as pd
import numpy as np

from preproc_functions import *
from constants import *


In [4]:
df_proc = pd.read_csv('%s/PROCEDURES_ICD.csv' % MIMIC_3_DIR)
df_diag = pd.read_csv('%s/DIAGNOSES_ICD.csv' % MIMIC_3_DIR)

df_diag['ABS_CODE'] = df_diag.apply(lambda row: str(reformat(str(row[4]), True)), axis=1)
df_proc['ABS_CODE'] = df_proc.apply(lambda row: str(reformat(str(row[4]), False)), axis=1)

df_codes = pd.concat([df_diag, df_proc])

df_notes = pd.read_csv('%s/NOTEEVENTS.csv' % MIMIC_3_DIR)

/Users/nabeelnauman/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
# Only consider discharge summaries:

df_notes = df_notes[df_notes['CATEGORY'] == 'Discharge summary']

In [16]:
# Drop rows with null HADM_IDs

df_notes = df_notes.dropna(subset=['HADM_ID'])

In [17]:
# Convert all IDs to int

df_notes['SUBJECT_ID'] = df_notes['SUBJECT_ID'].apply(lambda x: int(x))
df_notes['HADM_ID'] = df_notes['HADM_ID'].apply(lambda x: int(x))

df_codes['SUBJECT_ID'] = df_codes['SUBJECT_ID'].apply(lambda x: int(x))
df_codes['HADM_ID'] = df_codes['HADM_ID'].apply(lambda x: int(x))


In [18]:
# Some non-aggressive preprocessing. More preprocessing will be done by model tokenizer.

df_notes = preproc(df_notes)

In [19]:
# Append all notes and addenda

df_notes_grouped = df_notes.groupby(by=['HADM_ID','SUBJECT_ID'], as_index=False).agg({'TEXT': lambda x: ' '.join(x)})

In [20]:
df_codes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ABS_CODE
0,1297,109,172335,1.0,40301,403.01
1,1298,109,172335,2.0,486,486
2,1299,109,172335,3.0,58281,582.81
3,1300,109,172335,4.0,5855,585.5
4,1301,109,172335,5.0,4254,425.4


In [21]:
# Left join the two dataframes such that we discard any HADM_IDs not appearing in df_notes

merged = pd.merge(df_notes_grouped, df_codes, on='HADM_ID', how='left')


In [22]:
# Group by HADM_ID and SUBJECT_ID, aggregating on the ICD9_codes and appending them to a list.

grouped = merged.groupby(by=['HADM_ID', 'SUBJECT_ID_x'], as_index=False).agg({'ABS_CODE': lambda x: list(x),
                                                                              'TEXT': 'first'})

In [23]:
# Just some cosmetic stuff

grouped = grouped.sort_values(['SUBJECT_ID_x', 'HADM_ID'])
grouped = grouped.rename(columns={'ABS_CODE': 'LABELS', 'SUBJECT_ID_x': 'SUBJECT_ID'})
grouped = grouped[['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS']]
grouped = grouped.reset_index(drop=True)

In [26]:
# Convert labels to a semi-colon seperated string.

grouped['LABELS'] = grouped['LABELS'].apply(lambda x: ';'.join(x))

In [2]:
import pandas as pd

df = pd.read_pickle('dataframes/df_data.pkl')

In [3]:
df.head()

,SUBJECT_ID,HADM_ID,TEXT,LABELS
0,3,145834,Admission Date: Discharge Date: Date ...,038.9;785.59;584.9;427.5;410.71;428.0;682.6;42...
1,4,185777,Admission Date: Discharge Date: Date ...,042;136.3;799.4;276.3;790.7;571.5;041.11;V09.0...
2,6,107064,Admission Date: Discharge Date: Dat...,403.91;444.0;997.2;276.6;276.7;285.9;275.3;V15...
3,9,150750,Admission Date: Discharge Date: Date...,431;507.0;428.0;584.9;276.5;401.9;96.72;96.04
4,10,184167,Admission Date: Discharge Date: Dat...,V30.00;774.2;765.25;765.15;V29.0;99.83;99.15;96.6


In [5]:
import pandas as pd

df = pd.read_pickle('dataframes/df_data.pkl')

In [48]:
df.iloc[50].TEXT

"Admission Date:                Discharge Date:      Service: MEDICINE  Allergies: No Drug Allergy Information on File  Attending: Chief Complaint: right hip pain, fall  Major Surgical or Invasive Procedure: Intubation   History of Present Illness: This is a 88 year-old man with dementia, COPD, CHF, osteoarthritis s/p L hip replacement and h/o TIA who presented to the ED after a fall. In the ED, he was initially stable, but he began to become increasingly agitated and aggressive. He received haloperidol and ativan, but remained combative. His O2 sats never dipped below 90% but he became even more agitated, diaphoretic and, after discussion with the pt's daughter, the decision was made to intubate him so a w/u of his fall could be undertaken.  There was some concern about O2 sats in the low 90s, and after a d-dimer returned at 1000, a CTPA was done. It was negative. A head CT revealed no acute change. Hip films revealed.  He was admitted to the ICU intubated.   Past Medical History: Dem

In [54]:
remove = ['admission date:', 'discharge date:', 'date of birth:', 'service:', 'chief complaint:', 'HISTORY OF PRESENT ILLNESS:',
          'PAST MEDICAL HISTORY:', 'admission diagnosis:', 'history of the present illness:', 'attending:', 'cc:']





In [55]:
# Remove those AND start at token 12:

remove

['admission date:',
 'discharge date:',
 'date of birth:',
 'service:',
 'chief complaint:',
 'history of present illness:',
 'past medical history:']

In [60]:
import re

df['TEXT'] = df['TEXT'].apply(lambda x: re.sub(r'|'.join(map(re.escape, remove)), '', x.lower()))

In [62]:
df['TEXT'] = df['TEXT'].apply(lambda x: x.lstrip())

In [71]:
df.iloc[9].TEXT

"sex:   f   cardiothoracic  allergies: bactrim / ampicillin / remeron  attending:  history of stroke  major surgical or invasive procedure:  minimally invasive closure of patent foramen ovale    mrs.  is a 47 year old female who suffered a cerebellar stroke in . workup at that time revealed a patent foramen ovale. she is currently followed by dr. (neurologist) from the . full hypercoagulability workup was unremarkable. since , she has had no other neurological events. in preperation for surgical intervention, she underwent cardiac catheterization in  which showed normal coronary arteries and normal left ventricular function.   patent foramen ovale; history of stroke/tia; depression; anxiety; borderline hyperlipidemia; herniation of cervical discs; patella-femoral syndrome; s/p bunionectomies  social history: denies tobacco. admits to occasional etoh. she is an employee of the  in the neuro-pysch department. she is married with two children. she denies ivda and recreational drugs.  fami

In [73]:
df.to_pickle('dataframes/df_v2.pkl')

In [74]:
df

,SUBJECT_ID,HADM_ID,TEXT,LABELS
0,3,145834,sex: m medicine admitted from rehabilit...,038.9;785.59;584.9;427.5;410.71;428.0;682.6;42...
1,4,185777,sex: f shortness of breath and fevers. ...,042;136.3;799.4;276.3;790.7;571.5;041.11;V09.0...
2,6,107064,sex: f admission diagnosis: end stage re...,403.91;444.0;997.2;276.6;276.7;285.9;275.3;V15...
3,9,150750,"sex: m neurology weakness, inability to...",431;507.0;428.0;584.9;276.5;401.9;96.72;96.04
4,10,184167,"sex: f baby girl is a 1,385 gram, forme...",V30.00;774.2;765.25;765.15;V29.0;99.83;99.15;96.6
5,11,194540,sex: f neurosurgery allergies: penicillin...,191.3;15.9;11.3;11.3;92.29;99.25
6,12,112213,sex: m surgery mr. is a 72-year-old m...,157.0;574.10;997.1;427.5;998.11;401.9;568.0;55...
7,13,143045,sex: f cardiac surgery chest pain. t...,414.01;411.1;250.00;401.9;272.0;36.12;36.15;39...
8,17,161087,sex: f cardiothoracic allergies: bactrim ...,423.9;511.9;785.51;458.9;311;722.0;719.46;272....
9,17,194023,sex: f cardiothoracic allergies: bactrim ...,745.5;458.29;V12.59;272.4;35.71;39.61;88.72


In [75]:
len(df)

52726